In [1]:
import pandas as pd
from UtilsCreateDataFrame import *
import numpy as np
from dataclasses import dataclass, asdict
import matplotlib.pyplot as plt
import numpy_ext as npe
from statsmodels.tsa.stattools import adfuller
from Strategy import *
from Regression import *
import datetime


In [2]:
path = "BTC_ETH_2021-01.csv.gz"
df_ = pd.read_csv(path)
df_ = df_.set_index(pd.to_datetime(df_.time))
df_.drop(columns = ["time"], inplace = True)
df_

,BTC,ETH
time,,
2021-01-01 00:00:00+00:00,28961.66,737.02
2021-01-01 00:01:00+00:00,29009.91,738.74
2021-01-01 00:02:00+00:00,28989.30,737.82
2021-01-01 00:03:00+00:00,28982.69,737.43
2021-01-01 00:04:00+00:00,28975.65,736.89
...,...,...
2021-01-31 23:55:00+00:00,33215.89,1316.72
2021-01-31 23:56:00+00:00,33103.66,1315.04
2021-01-31 23:57:00+00:00,33069.47,1313.72


In [16]:
# with 1minute frequency a day has 1440 observations
#let's try this on a week 
index_input = df_.index
x, y = df_.to_numpy()[:,0], df_.to_numpy()[:,1]
window = 1440
last = 1440*7
rolling = npe.rolling_apply(linearRegression_np, window, x[:last], y[:last], n_jobs = 4)
df_beta = ResultDataFrame(rolling, index_input[:last])
df_beta

,beta,intercept,r2,res_std,res_mean,stationarity_pvalue
time,,,,,,
2021-01-01 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-01 00:01:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-01 00:02:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-01 00:03:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-01 00:04:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2021-01-07 23:55:00+00:00,0.023285,332.990880,0.826404,10.520993,-1.173185e-13,0.117492
2021-01-07 23:56:00+00:00,0.023275,333.345485,0.826030,10.532041,6.805420e-14,0.112567
2021-01-07 23:57:00+00:00,0.023264,333.709493,0.825598,10.544829,3.615873e-14,0.093566


In [17]:
result_strategy = applyStrategyRolling(
    df_beta = df_beta,
    df_price = df_,
    trading_window_day = 1, 
    frequency = "1m", 
    thr_pval = 0.1
)

In [20]:
#total return 
for strat in result_strategy[0]:
    print(strat)
    for ret in strat.ret_:
        tot_ret *= ret
tot_ret

ResultStrategy(ret_=[0.9018894491767744], ret_type=['Short'], enter_=[Timestamp('2021-01-02 12:46:00+0000', tz='UTC')], exit_=[Timestamp('2021-01-03 11:45:00+0000', tz='UTC')])
ResultStrategy(ret_=[1.042344903002118, 1.0603532429601974, 1.0399594381609971], ret_type=['Short', 'Long', 'Short'], enter_=[Timestamp('2021-01-04 03:59:00+0000', tz='UTC'), Timestamp('2021-01-04 09:41:00+0000', tz='UTC'), Timestamp('2021-01-05 00:15:00+0000', tz='UTC')], exit_=[Timestamp('2021-01-04 08:43:00+0000', tz='UTC'), Timestamp('2021-01-04 14:01:00+0000', tz='UTC'), Timestamp('2021-01-05 03:45:00+0000', tz='UTC')])
ResultStrategy(ret_=[0.9915151846319464], ret_type=['Long'], enter_=[Timestamp('2021-01-05 19:26:00+0000', tz='UTC')], exit_=[Timestamp('2021-01-06 03:47:00+0000', tz='UTC')])
ResultStrategy(ret_=[1.0573447240830056, 1.0663836204712462, 0.9790145708543779], ret_type=['Long', 'Long', 'Short'], enter_=[Timestamp('2021-01-06 03:50:00+0000', tz='UTC'), Timestamp('2021-01-06 05:45:00+0000', tz='U

1.742579693799463

In [19]:
result_strategy[1]

[beta                   9.511356e-03
 intercept              4.517162e+02
 r2                     1.762130e-01
 res_std                4.590768e+00
 res_mean               6.900159e-14
 stationarity_pvalue    9.486289e-02
 Name: 2021-01-02 11:44:00+00:00, dtype: float64,
 beta                  -6.464830e-02
 intercept              3.057470e+03
 r2                     2.547148e-01
 res_std                6.392163e+01
 res_mean              -7.124375e-13
 stationarity_pvalue    9.625794e-02
 Name: 2021-01-04 03:44:00+00:00, dtype: float64,
 beta                   3.946798e-02
 intercept             -2.234963e+02
 r2                     6.467541e-01
 res_std                2.645552e+01
 res_mean               9.221266e-14
 stationarity_pvalue    1.397301e-02
 Name: 2021-01-05 03:46:00+00:00, dtype: float64,
 beta                   2.529187e-02
 intercept              2.327950e+02
 r2                     9.031174e-01
 res_std                9.882658e+00
 res_mean               3.836931e-14